### In this notebook, I first generate faces using captions from COCO dataset using ERNIE-ViLG, and then, I using face detector to extract faces from the images generated. 

In [4]:
#!pip install translators --upgrade

  Using cached translators-5.5.0-py3-none-any.whl (30 kB)
  Using cached PyExecJS-1.5.1-py3-none-any.whl
  Using cached lxml-4.9.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.9 MB)
  Using cached pathos-0.3.0-py3-none-any.whl (79 kB)
  Using cached pox-0.3.2-py3-none-any.whl (29 kB)
  Using cached ppft-1.7.6.6-py3-none-any.whl (52 kB)
  Using cached multiprocess-0.70.14-py38-none-any.whl (132 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.12.2
    Uninstalling multiprocess-0.70.12.2:
      Successfully uninstalled multiprocess-0.70.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pad

Load caption data

In [39]:
import paddlehub as hub
import translators as ts
lines = open('face_captions.txt').read().splitlines()

In [43]:
import random
random.seed(42)
#Generate 1000 random numbers between 0 and 9999
# I plan to generate about 600 images, but may need to come back generate more if some of the images do not contain a good face
randomlist = random.sample(range(0, 9999), 1100)

In [44]:
len(randomlist)

1100

In [45]:
randomlist[:10]

[1824, 409, 4506, 4012, 3657, 2286, 1679, 8935, 1424, 9674]

Start generating face images

In [46]:
import paddlehub as hub
import translators as ts
module = hub.Module(name="ernie_vilg")
for i in range(1028,1029):
    prompt = ts.baidu(lines[randomlist[i]], professional_field='common')
    try: # for certain prompts, error message "暂不支持创作该内容，请修改后再试 (does not support this prompt, please modify)" was returned
        module.generate_image(text_prompts=prompt, style='写实风格',  output_dir='./generated_face_images_with_ERNIE-ViLG/',topk=1)
        print(i)      
    except:
        continue

  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
1028


In [34]:
i

1027

extract faces from the generated face images 

In [52]:
#set up folders for extracted faces 
# initialize COCO API for instance annotations
import os
dataDir = './'

image_folder = "extracted_generated_faces"

savedirimage = dataDir + image_folder



if not os.path.exists(savedirimage):
	os.makedirs(savedirimage)

In [53]:
# install all packages
!pip install keras
!pip install tensorflow
from keras.models import load_model
!pip install mtcnn
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

import numpy as np
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
import matplotlib.pyplot as plt


# Simple python package to shut up Tensorflow warnings and logs.
!pip install silence_tensorflow
import silence_tensorflow.auto

0.1.0


I did not use threshold as filtering criteria because this will gaurantee low FID score later. I still kept the dimension requirement. 

In [54]:
# extract a single face from a given photograph
def extract_face(filename, index, required_size=(250, 250)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    if len(results) >=1:
        x1, y1, width, height = results[0]['box']
        if height-width>=15:
            # bug fix
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            # extract the face
            face = pixels[y1:y2, x1:x2]
            # resize pixels to the model size
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array = np.asarray(image)
            plt.imsave(os.path.join(savedirimage, 'face_img_{}.jpg'.format(index)),face_array)
            return True
    return False

In [55]:
files = os.listdir('generated_face_images_with_ERNIE-ViLG/')

In [57]:
for file in files:
    print(file)

.ipynb_checkpoints
一名女网球运动员准备击球。_0.png


In [59]:
index =0
for file in files:
    if extract_face(os.path.join('./generated_face_images_with_ERNIE-ViLG/',file), index = index):
        print(index)
        index+=1
        
    

1/1 [==============================] - 0s 152ms/step
697
